In [8]:
import pandas as pd
from dotenv import load_dotenv
load_dotenv(override=True)
df = pd.read_csv('../curated/DEVRT-DACIA-SPRING.csv')

In [16]:
from data_analysis_agent.automotive_data_quality import generate_automotive_quality_report

In [30]:
# Final demonstration: Silent mode when writing to files
print("Running automotive quality report with JSON output...")
generate_automotive_quality_report(df, json_output_file='report.json')
print("Report completed! Check report.json for results.")

INFO:data_analysis_agent.automotive_data_quality:JSON report saved to report.json


Running automotive quality report with JSON output...
Report completed! Check report.json for results.


In [31]:
df_questions = pd.read_json('../curated/15_DEVRT-DACIA-SPRING-questions.jsonl', lines=True)
df_answers = pd.read_json('../curated/15_DEVRT-DACIA-SPRING-labels.jsonl', lines=True)

In [32]:
df_questions['question'].head(), df_answers['common_answers'].head()

(0    What is the correlation coefficient between al...
 1    What is the average altitude I drive at? Fill ...
 2    Which drivers achieve the best energy efficien...
 3    What is the average state of health (SOH) perc...
 4    What is the correlation coefficient between fr...
 Name: question, dtype: object,
 0        {'altitude_speed_correlation': -0.03}
 1           {'average_altitude_meters': 135.1}
 2    {'efficient_drivers': ['d1', 'd2', 'd3']}
 3                  {'fleet_average_soh': 98.5}
 4    {'frontal_wind_speed_correlation': 0.057}
 Name: common_answers, dtype: object)

In [34]:
from openai import OpenAI


# 1) Upload the JSON and create a vector store
client = OpenAI()  # requires OPENAI_API_KEY

upload_report = client.files.create(
    file=open("report.json", "rb"),
    purpose="user_data",
)

upload_csv = client.files.create(
    file=open("../curated/DEVRT-DACIA-SPRING.csv", "rb"),
    purpose="user_data",
)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"


In [35]:
upload_report, upload_csv

(FileObject(id='file-MUUbFY99D7otoy1VxvfcUR', bytes=69264, created_at=1754725273, filename='report.json', object='file', purpose='user_data', status='processed', expires_at=None, status_details=None),
 FileObject(id='file-UEosua5u86rkKhVexb1uLX', bytes=3006479, created_at=1754725274, filename='DEVRT-DACIA-SPRING.csv', object='file', purpose='user_data', status='processed', expires_at=None, status_details=None))

In [60]:
system_prompt = (
"You are an automotive data analysis expert with access to a code execution environment. "
"You will receive two files: "
"1) A CSV file containing automotive telemetry data with sensor readings, timestamps, and vehicle parameters "
"2) A JSON report containing comprehensive data quality assessment including missing values, outliers, signal validation, and statistical summaries. "
"Use the code interpreter to read, analyze, and visualize this data. "
"When answering questions: "
"- Always examine both files programmatically to understand the data structure "
"- Reference specific findings from the quality report when relevant "
"- Create visualizations and statistical analyses as needed "
"- Provide automotive domain-specific insights about signal patterns, anomalies, and data integrity "
"- If you need to perform calculations or data transformations, write and execute code "
"- Be precise and cite specific data points from your analysis "
"Answer accurately based on the data; if something is unknown or unclear, say so explicitly."
)

In [88]:
response = client.responses.create(
    model="gpt-5",
    tools=[{
      "type": "code_interpreter",
      "container": {"type": "auto",
                    "file_ids": [upload_csv.id, upload_report.id]}
    }],
    input=[
        {   
            "role": "developer",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": df_questions['question'][7],
                },
            ]
        }
    ]
)

print(response.output_text)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


309.75


In [86]:
from openai.types.responses import ResponseCodeInterpreterToolCall
print(''.join([item.code for item in response.output if isinstance(item, ResponseCodeInterpreterToolCall)]))

import json, pandas as pd, numpy as np, math, os, textwrap, sys, statistics, re, warningscsv_path = "/mnt/data/file-UEosua5u86rkKhVexb1uLX-DEVRT-DACIA-SPRING.csv"
json_path = "/mnt/data/file-MUUbFY99D7otoy1VxvfcUR-report.json"

# Load files
try:
    df = pd.read_csv(csv_path)
except Exception as e:
    df = None
    csv_err = str(e)

try:
    with open(json_path, "r") as f:
        report = json.load(f)
except Exception as e:
    report = None
    json_err = str(e)

(type(df), len(df) if df is not None else None, type(report) if report is not None else None)# Inspect columns
cols = list(df.columns) if df is not None else []
colsimport pandas as pd
import numpy as np

df_work = df.copy()

# Ensure numeric types
for col in ["speedAvg", "soh"]:
    if col in df_work.columns:
        df_work[col] = pd.to_numeric(df_work[col], errors="coerce")

# Fill missing speedAvg with mean
if "speedAvg" in df_work.columns:
    mean_speed = df_work["speedAvg"].mean(skipna=True)
    df_work["speedAvg"] =

In [ ]:
import json, pandas as pd, numpy as np, math, os, textwrap, sys, statistics, re, warningscsv_path = "/mnt/data/file-UEosua5u86rkKhVexb1uLX-DEVRT-DACIA-SPRING.csv"
json_path = "/mnt/data/file-MUUbFY99D7otoy1VxvfcUR-report.json"

# Load files
try:
    df = pd.read_csv(csv_path)
except Exception as e:
    df = None
    csv_err = str(e)

try:
    with open(json_path, "r") as f:
        report = json.load(f)
except Exception as e:
    report = None
    json_err = str(e)

(type(df), len(df) if df is not None else None, type(report) if report is not None else None)# Inspect columns
cols = list(df.columns) if df is not None else []
colsimport pandas as pd
import numpy as np

df_work = df.copy()

# Ensure numeric types
for col in ["speedAvg", "soh"]:
    if col in df_work.columns:
        df_work[col] = pd.to_numeric(df_work[col], errors="coerce")

# Fill missing speedAvg with mean
if "speedAvg" in df_work.columns:
    mean_speed = df_work["speedAvg"].mean(skipna=True)
    df_work["speedAvg"] = df_work["speedAvg"].fillna(mean_speed)

# Define bins and labels
bins = [0, 50, 80, 100, np.inf]
labels = ["0-50 km/h", "50-80 km/h", "80-100 km/h", "100+ km/h"]

# Assign bins
# We consider only non-negative speeds; negative speeds will be clipped to 0 for binning
if "speedAvg" in df_work.columns:
    speeds = df_work["speedAvg"].clip(lower=0)
    df_work["speed_bin"] = pd.cut(speeds, bins=bins, labels=labels, right=False, include_lowest=True)

# Compute average SOH per bin
if "soh" in df_work.columns and "speed_bin" in df_work.columns:
    soh_by_bin = df_work.groupby("speed_bin")["soh"].mean()
    # Round to 2 decimals
    soh_by_bin_rounded = soh_by_bin.round(2)
    soh_by_bin, soh_by_bin_rounded, mean_speed, df_work["speed_bin"].isna().sum()
else:
    soh_by_bin = None# Let's compute and return the series to view
soh_by_bin_dict = None
soh_by_bin_rounded_dict = None
mean_speed_val = None
n_na_speed_bin = None

if "soh" in df_work.columns and "speed_bin" in df_work.columns:
    soh_by_bin = df_work.groupby("speed_bin")["soh"].mean()
    soh_by_bin_rounded = soh_by_bin.round(2)
    soh_by_bin_dict = soh_by_bin.to_dict()
    soh_by_bin_rounded_dict = soh_by_bin_rounded.to_dict()
    mean_speed_val = float(df_work["speedAvg"].mean())
    n_na_speed_bin = int(df_work["speed_bin"].isna().sum())

soh_by_bin_dict, soh_by_bin_rounded_dict, mean_speed_val, n_na_speed_bin# Select bin with highest SOH based on rounded means
if soh_by_bin_dict is not None:
    soh_rounded = pd.Series(soh_by_bin_rounded_dict)
    # Ensure ordering by our labels
    soh_rounded = soh_rounded.reindex(labels)
    optimal_bin = soh_rounded.idxmax() if not soh_rounded.dropna().empty else None
else:
    optimal_bin = None

optimal_bin

In [ ]:
from openai import OpenAI
client = OpenAI()

deleted_csv = client.files.delete(upload_csv.id)

In [ ]:
deleted_report = client.files.delete(upload_report.id)